In [8]:
import requests
from bofire.data_models.domain.api import Domain

from pydantic import BaseModel, Field
from bofire.data_models.strategies.api import AnyStrategy, RandomStrategy, SoboStrategy
from typing import Optional
from bofire.data_models.features.api import ContinuousInput

from bofire.data_models.dataframes.api import Experiments, Candidates
from bofire.benchmarks.api import Himmelblau, DTLZ2
import json


class CandidateRequest(BaseModel):
    strategy_data: AnyStrategy
    n_candidates: int = Field(
        default=1, gt=0, description="Number of candidates to generate"
    )
    experiments: Optional[Experiments]
    pendings: Optional[Candidates]

bench = Himmelblau()
bench2 = DTLZ2(dim=6)
experiments = bench.f(bench.domain.inputs.sample(10), return_complete=True)
experiments2 = bench2.f(bench2.domain.inputs.sample(10), return_complete=True)


cr = CandidateRequest(
    strategy_data=SoboStrategy(domain=bench.domain),
    n_candidates=1,
    experiments=Experiments.from_pandas(experiments2, bench2.domain),
    pendings=None
)


URL = "http://127.0.0.1:8000/candidates"
HEADERS = {'accept': 'application/json', 'Content-Type': 'application/json'}

response = requests.post(url=f"{URL}/generate", data=cr.model_dump_json(), headers=HEADERS)


In [10]:
response.content

b'{"detail":[{"type":"value_error","loc":["body"],"msg":"Value error, no col for input feature `y`","input":{"strategy_data":{"type":"SoboStrategy","domain":{"type":"Domain","inputs":{"type":"Inputs","features":[{"type":"ContinuousInput","key":"x_1","unit":null,"bounds":[-6.0,6.0],"local_relative_bounds":null,"stepsize":null},{"type":"ContinuousInput","key":"x_2","unit":null,"bounds":[-6.0,6.0],"local_relative_bounds":null,"stepsize":null}]},"outputs":{"type":"Outputs","features":[{"type":"ContinuousOutput","key":"y","unit":null,"objective":{"type":"MinimizeObjective","w":1.0,"bounds":[0.0,1.0]}}]},"constraints":{"type":"Constraints","constraints":[]}},"seed":null,"num_restarts":8,"num_raw_samples":1024,"maxiter":2000,"batch_limit":8,"descriptor_method":"EXHAUSTIVE","categorical_method":"EXHAUSTIVE","discrete_method":"EXHAUSTIVE","surrogate_specs":{"surrogates":[{"hyperconfig":{"type":"SingleTaskGPHyperconfig","hyperstrategy":"FactorialStrategy","inputs":{"type":"Inputs","features":[{"

In [28]:
Candidates(**json.loads(response.content)).to_pandas()

,x_1,x_2,y_pred,y_std,y_des
0,-3.029691,-1.247332,45.611653,250.717458,-45.611653
